# Factorización LU secuencial

## Algoritmo de Doolittle

In [52]:
using LinearAlgebra
function LU(A::Matrix{Float64})
    m,n = size(A)
    L = Matrix{Float64}(I, n, n)
    U = copy(A)
    for j = 1:n-1
        for i = j+1:n
            L[i,j] = U[i,j]/U[j,j]
            U[i,j:end] -= L[i,j]*U[j,j:end]
        end
    end
    return L,U
end

LU

### Ejemplo matriz 3x3 simple

In [56]:
A = [1.0 2.0 3.0; 4.0 5.0 6.0; 7.0 8.0 10.0]
L, U = LU(A)
display(L)
display(U)
display(L*U)

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 4.0  1.0  0.0
 7.0  2.0  1.0

3×3 Matrix{Float64}:
 1.0   2.0   3.0
 0.0  -3.0  -6.0
 0.0   0.0   1.0

3×3 Matrix{Float64}:
 1.0  2.0   3.0
 4.0  5.0   6.0
 7.0  8.0  10.0

## Factorización LU paralelo

In [1]:
using LinearAlgebra
using Base.Threads

function parallel_LU(A::Matrix{Float64})
    m,n = size(A)
    L = Matrix{Float64}(I, n, n)
    U = copy(A)
    num_threads = Threads.nthreads()
    threads = []
    for j = 1:n-1
        for i = j+1:n
            if length(threads) < num_threads
                push!(threads, Threads.@spawn begin
                    L[i,j] = U[i,j]/U[j,j]
                    U[i,j:end] -= L[i,j]*U[j,j:end]
                end)
            else
                wait(threads[1])
                popfirst!(threads)
            end
        end
    end
    for thread in threads
        wait(thread)
    end
    return L,U
end

parallel_LU (generic function with 1 method)

### Ejemplo matriz 3x3 simple

In [65]:
A = [1.0 2.0 3.0; 4.0 5.0 6.0; 7.0 8.0 10.0]
L, U = parallel_LU(A)
display(L)
display(U)
display(L*U)

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 4.0  1.0  0.0
 7.0  1.6  1.0

3×3 Matrix{Float64}:
 1.0    2.0    3.0
 0.0   -3.0   -6.0
 0.0  -14.0  -20.6

3×3 Matrix{Float64}:
 1.0   2.0   3.0
 4.0   5.0   6.0
 7.0  -4.8  -9.2

## Factorización LU por bloques 

In [7]:
using LinearAlgebra

function is_singular(A)
    return det(A) == 0
end

function block_LU(A)
    n = size(A, 1)
    if n == 1
        return A, A, A
    end

    m = n ÷ 2
    A11 = A[1:m, 1:m]
    A12 = A[1:m, m+1:end]
    A21 = A[m+1:end, 1:m]
    A22 = A[m+1:end, m+1:end]

    if is_singular(A11) || is_singular(A22)
        error("La matriz es singular")
    end

    L11, U11 = impLU(A11)
    U12 = L11 \ A12
    L21 = A21 / U11
    L22, U22 = impLU(A22 - L21*U12)

    L = [L11 zeros(m, n-m); L21 L22]
    U = [U11 U12; zeros(n-m, m) U22]

    return L, U
end
function impLU(A::Matrix{Float64})
    m,n = size(A)
    L = Matrix{Float64}(I, n, n)
    U = copy(A)
    for j = 1:n-1
        for i = j+1:n
            L[i,j] = U[i,j]/U[j,j]
            U[i,j:end] -= L[i,j]*U[j,j:end]
        end
    end
    return L,U
end

impLU (generic function with 1 method)

### Pruebas

In [8]:
A = rand(4096,4096)
L, U = block_LU(A)

([1.0 0.0 … 0.0 0.0; 4.676563253466478 1.0 … 0.0 0.0; … ; 0.590046977026757 -0.2104368002745426 … 1.0 0.0; 4.781248571768607 1.2616612400521499 … 0.3836332357057343 1.0], [0.18843814628404998 0.42932330950033626 … 0.16636107792096222 0.7117202075850602; 1.1102230246251565e-16 -1.0813293747232184 … -0.021141794899128485 -2.827269698434805; … ; 0.0 0.0 … -49.822681562225846 -85.15664242962262; 0.0 0.0 … 0.0 -23.576608339536662])

## Factorización LU por bloques con LU paralelo

In [4]:
using Base.Threads
using LinearAlgebra

function is_singular(A)
    return det(A) == 0
end

function pseudo_parallel_LU_block_LU(A)
    n = size(A, 1)
    if n == 1
        return A, A, A
    end

    m = n ÷ 2
    A11 = A[1:m, 1:m]
    A12 = A[1:m, m+1:end]
    A21 = A[m+1:end, 1:m]
    A22 = A[m+1:end, m+1:end]

    if is_singular(A11) || is_singular(A22)
        error("La matriz es singular")
    end

    L11, U11 = parallel_LU(A11)
    U12 = L11 \ A12
    L21 = A21 / U11
    L22, U22 = parallel_LU(A22 - L21*U12)

    L = [L11 zeros(m, n-m); L21 L22]
    U = [U11 U12; zeros(n-m, m) U22]

    return L, U
end

function parallel_LU(A::Matrix{Float64})
    m,n = size(A)
    L = Matrix{Float64}(I, n, n)
    U = copy(A)
    num_threads = Threads.nthreads()
    threads = []
    for j = 1:n-1
        for i = j+1:n
            if length(threads) < num_threads
                push!(threads, Threads.@spawn begin
                    L[i,j] = U[i,j]/U[j,j]
                    U[i,j:end] -= L[i,j]*U[j,j:end]
                end)
            else
                wait(threads[1])
                popfirst!(threads)
            end
        end
    end
    for thread in threads
        wait(thread)
    end
    return L,U
end

parallel_LU (generic function with 1 method)

### Pruebas

In [5]:
A = rand(4096,4096)
L, U = pseudo_parallel_LU_block_LU(A)

([1.0 0.0 … 0.0 0.0; 0.24873496849292703 1.0 … 0.0 0.0; … ; -8.91709715334129 -12.464606627411785 … 1.0 0.0; -1.848266003868957 -3.8976563469118632 … -0.8134867399098984 1.0], [0.9749886829104764 0.8935188148175884 … 0.8466022479749088 0.3364897449968206; 0.0 0.3885563330314892 … 0.598434529602937 0.29014922503996354; … ; 0.0 0.0 … -4052.2365091379615 -7012.950644595873; 0.0 0.0 … -3211.977320295415 -5559.952399560551])

In [1]:
print(Threads.nthreads())

4